In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import manifold
import ipywidgets as widgets
#from IPython.display import display
#from mpl_toolkits.mplot3d import Axes3D

In [2]:
##################
### Similarity ###
##################

# Set range so it's the same for all plots
rng = range(0, 11)

# Load data
sim_all = pd.read_csv("data/similarity_data_averaged.csv")

# Create dropdown menu
cat_options = sorted(sim_all["category"].unique())
dropdown = widgets.Dropdown(options=cat_options, value=cat_options[0], description="Category:")

# Udate plot for each category
def update_plot(category):
    plt.rcParams['figure.facecolor'] = 'white'
    temp = sim_all[sim_all["category"] == category]
    temp_pivot = temp.pivot(index="sound1", columns="sound2", values="response")

    #temp_pivot = temp.pivot("sound1", "sound2", "response")
    plt.figure(figsize=(8, 8))
    ax = sns.heatmap(data=temp_pivot, annot=True, cmap="Spectral_r", square=True)
    ax.set_xlabel("Sound 1")
    ax.set_ylabel("Sound 2")
    ax.set_title(category)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.set_ylim(ax.get_ylim()[::-1]) # flip y-axis
    cbar = ax.collections[0].colorbar
    cbar.set_ticks(np.linspace(rng[0], rng[-1], num=11))
    cbar.set_ticklabels(np.linspace(rng[0], rng[-1], num=11))
    cbar.ax.set_ylabel("Response", rotation=-90, va="bottom")
    plt.show()

# Display plot
widgets.interact(update_plot, category=dropdown)
#update_plot(dropdown.value)


interactive(children=(Dropdown(description='Category:', options=('BirdSong', 'BirdSquawk', 'BirdsFlyingOff', '…

<function __main__.update_plot(category)>

In [6]:
###########
### MDS ###
###########

# Load data
similarity_data = pd.read_csv("data/similarity_data.csv")

# Udate plot for each category
def update_plot(category):
    plt.rcParams['figure.facecolor'] = 'white'
    data = similarity_data[similarity_data["category"] == category]

    # Compute mean response for each sound pair
    data = data.groupby(["sound1", "sound2"]).mean().reset_index()

    # Create a dictionary that maps each sound to an integer
    sound_to_int = {sound: i for i, sound in enumerate(data["sound1"].unique())}

    # Compute dissimilarity matrix
    n_sounds = len(sound_to_int)
    dissimilarity_matrix = np.zeros((n_sounds, n_sounds))
    for _, row in data.iterrows():
        i = sound_to_int[row["sound1"]]
        j = sound_to_int[row["sound2"]]
        dissimilarity_matrix[i, j] = 1 - row["response"]
        dissimilarity_matrix[j, i] = 1 - row["response"]

    # Perform MDS in 2D and 3D
    mds_seed = 123
    mds_2d = manifold.MDS(n_components=2, dissimilarity="precomputed", random_state=mds_seed)
    mds_3d = manifold.MDS(n_components=3, dissimilarity="precomputed", random_state=mds_seed)
    coordinates_2d = mds_2d.fit_transform(dissimilarity_matrix)
    coordinates_3d = mds_3d.fit_transform(dissimilarity_matrix)

    # Plot MDS in 2D
    plt.figure(figsize=(8, 8))
    plt.scatter(coordinates_2d[:, 0], coordinates_2d[:, 1])
    plt.scatter(np.mean(coordinates_2d[:, 0]), np.mean(coordinates_2d[:, 1]), marker='*', s=200, color='red')
    for i, label in enumerate(sound_to_int.keys()):
        plt.annotate(label, (coordinates_2d[i, 0], coordinates_2d[i, 1]))
    plt.axis("equal")
    plt.title(f"MDS Plot in 2D for {category}")
    plt.show()

    # Plot MDS in 3D
    figstat = plt.figure(figsize=(8, 8))
    ax = figstat.add_subplot(111, projection="3d")
    ax.scatter(coordinates_3d[:, 0], coordinates_3d[:, 1], coordinates_3d[:, 2])
    ax.scatter(np.mean(coordinates_3d[:, 0]), np.mean(coordinates_3d[:, 1]), np.mean(coordinates_3d[:, 2]), marker='*', s=200, color='red')
    for i, label in enumerate(sound_to_int.keys()):
        ax.text(coordinates_3d[i, 0], coordinates_3d[i, 1], coordinates_3d[i, 2], label)
    ax.set_title(f"MDS Plot in 3D for {category}")
    plt.show()

# Add dropdown menu
cat_options = sorted(similarity_data["category"].unique())
category_dropdown = widgets.Dropdown(options=cat_options, value=cat_options[0], description='Category:')
output = widgets.Output()

# Handle dropdown menu
def dropdown_eventhandler(change):
    output.clear_output()
    with output:
        update_plot(change.new)

# Attach event handler to dropdown
category_dropdown.observe(dropdown_eventhandler, names='value')

# Display dropdown and plot
display(category_dropdown)
display(output)


Dropdown(description='Category:', options=('BirdSong', 'BirdSquawk', 'BirdsFlyingOff', 'CarHorn', 'CarSkidding…

Output()

In [4]:
################
### Goodness ###
################

# Load data
good_summary = pd.read_csv("data/goodness_data.csv")

# Add dropdown menu
cat_options = sorted(good_summary["category"].unique())
dropdown_cat = widgets.Dropdown(options=cat_options, description='Select a category')

# Update figure for each category
def update_plot(category):
    plt.rcParams['figure.facecolor'] = 'white'
    temp = good_summary[good_summary["category"] == category]
    temp = temp.groupby("sound1")["response"].agg(["mean", "sem"]).reset_index()
    plt.figure(figsize=(8, 8))
    plt.bar(temp["sound1"], temp["mean"], yerr=temp["sem"], color="darkslateblue", capsize=4)
    plt.ylim(0, 10)
    plt.xlabel("Sound")
    plt.ylabel("Average rating")
    plt.title(category)
    plt.xticks(range(1, 11)) # set x axis to show all numbers from 1 to 10
    plt.gca().set_aspect('equal', adjustable='box')
    plt.style.use('classic')
    plt.show()

# Display plot
widgets.interactive(update_plot, category=dropdown_cat)

interactive(children=(Dropdown(description='Select a category', options=('BirdSong', 'BirdSquawk', 'BirdsFlyin…